# **Índice de Sentimento V1**

## *Sentimentos*

In [1]:
import pandas as pd

sent = pd.read_csv('C:/Users/User/Documents/GitHub/Dev Wizards/quant/data/news/noticias_mercado financeiro_sentimentos.csv')
sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'POSITIVE', 'Score']
sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score Signal'] = sent.loc[sent['Sentimento'] == 'NEGATIVE', 'Score'] * -1
sent.loc[sent['Sentimento'] == 'NEUTRAL', 'Score Signal'] = 0
sent

,Data,Notícia,Sentimento,Score,Score Signal
0,2024-09-03,Unipar e MELVER estão com inscrições abertas p...,NEUTRAL,0.700441,0.000000
1,2024-09-03,Mercado financeiro hoje: Vale (VALE3) em busca...,NEGATIVE,0.515265,-0.515265
2,2024-09-03,Briga entre X e STF pode afastar investidores ...,NEGATIVE,0.850306,-0.850306
3,2024-09-03,Podcast | Fechamento de Mercado - 03/09/2024,NEUTRAL,0.614209,0.000000
4,2024-09-03,Comentario semanal - Perspectivas,NEUTRAL,0.610426,0.000000
...,...,...,...,...,...
30477,2014-09-08,Meu salário é de R$ 20 mil; como investir para...,NEUTRAL,0.717937,0.000000
30478,2014-09-08,» AMAZONAS ATUAL - Cartões pré-pagos avançam n...,POSITIVE,0.697899,0.697899
30479,2014-09-08,Loja de roupas para executivas fatura R$ 600 m...,POSITIVE,0.721196,0.721196
30480,2014-09-08,Investir em obras de arte é opção para quem já...,NEUTRAL,0.653174,0.000000


In [2]:
sentimentos_g = []
sentimentos_t = []
for data in sent['Data'].unique():
    dia = sent[sent['Data'] == data]
    if dia['Sentimento'].value_counts().shape[0] != 1:
        total_dia_g = dia[dia['Sentimento'] != 'NEUTRAL']['Score'].sum() + dia[dia['Sentimento'] == 'NEUTRAL']['Score'].sum() * 0.5
        total_dia_t = len(dia[dia['Sentimento'] != 'NEUTRAL']) + len(dia[dia['Sentimento'] == 'NEUTRAL']) * 0.5
        soma_dia = dia['Score Signal'].sum()
        sentimentos_g.append(soma_dia/total_dia_g)
        sentimentos_t.append(soma_dia/total_dia_t)
    else:
        media = dia['Score Signal'].mean()
        sentimentos_g.append(media)
        sentimentos_t.append(media)

In [3]:
sentimentos_df = pd.DataFrame(sentimentos_g, index=sent['Data'].unique())
sentimentos_df.columns = ['Sentimento Guigas']
sentimentos_df['Sentimento Theozão'] = sentimentos_t
sentimentos_df.describe()

,Sentimento Guigas,Sentimento Theozão
count,3637.000000,3637.000000
mean,-0.186246,-0.135754
std,0.363518,0.260218
min,-0.953159,-0.837898
25%,-0.450010,-0.314190
50%,-0.206619,-0.140980
75%,0.049112,0.031827
max,0.948822,0.775933


## *Índice de Sentimento*

In [47]:
vix = pd.read_excel('./quant/data/vix/IVol-BR.xls')
vix

,year,month,day,ivolbr
0,2011,8,1,21.287107
1,2011,8,2,23.567432
2,2011,8,3,27.325140
3,2011,8,4,NaN
4,2011,8,5,29.760391
...,...,...,...,...
2410,2022,4,25,27.050753
2411,2022,4,26,25.338543
2412,2022,4,27,23.995495
2413,2022,4,28,24.415993
